In [12]:
#parameter ranking and selection (using a random tree)
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns




In [ ]:
#Enzo sent me the directly preprocessed file because I don't have enough storage right now
#But you can just run the pipeline otherwise
df = pd.read_csv('../../knnData/BA_US_knn_text.csv')


                beer_name   beer_id                  brewery_name  brewery_id  \
0           Kupfer Kolsch  289320.0  Copper State Brewing Company     49595.0   
1        Northwestern Alt  289321.0  Copper State Brewing Company     49595.0   
2          One Cent Wheat  289319.0  Copper State Brewing Company     49595.0   
3  Platinum Coffee Blonde  289324.0  Copper State Brewing Company     49595.0   
4  Platinum Coffee Blonde  289324.0  Copper State Brewing Company     49595.0   

                 style  abv            user_id  appearance  aroma  palate  \
0               Kölsch  4.4       n2185.211743        2.50   4.00    4.00   
1              Altbier  4.6       n2185.211743        3.00   3.75    4.00   
2              Witbier  5.4       n2185.211743        3.75   3.25    3.75   
3  American Blonde Ale  7.3       n2185.211743        3.25   3.75    3.50   
4  American Blonde Ale  7.3  angerhaus.1049226        4.00   4.25    3.75   

   taste  overall  rating text   avg      user_sta

In [3]:
df.head(3)

,beer_name,beer_id,brewery_name,brewery_id,style,abv,user_id,appearance,aroma,palate,taste,overall,rating,text,avg,user_state,beer_state
0,Kupfer Kolsch,289320.0,Copper State Brewing Company,49595.0,Kölsch,4.4,n2185.211743,2.50,4.00,4.00,3.75,3.75,3.76,NaN,3.76,North Carolina,Wisconsin
1,Northwestern Alt,289321.0,Copper State Brewing Company,49595.0,Altbier,4.6,n2185.211743,3.00,3.75,4.00,3.50,3.50,3.58,NaN,3.58,North Carolina,Wisconsin
2,One Cent Wheat,289319.0,Copper State Brewing Company,49595.0,Witbier,5.4,n2185.211743,3.75,3.25,3.75,3.50,3.50,3.48,NaN,3.48,North Carolina,Wisconsin


In [ ]:
#features to drop and features to keep for parameter selection
#Dropping user_id assumes that each review is independent of who wrote it
#Disregards the fact that user can write multiple reviews
columns_to_drop = ['user_id','beer_id', 'beer_name', 'brewery_name', 'brewery_id', 'avg', 'user_state', 'beer_state', 'text']

# Drop the specified columns

X = df.drop(columns=columns_to_drop)
y = df['user_state']

# Encode the target variable (user_state)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
X.head(3)

,style,abv,appearance,aroma,palate,taste,overall,rating
0,Kölsch,4.4,2.50,4.00,4.00,3.75,3.75,3.76
1,Altbier,4.6,3.00,3.75,4.00,3.50,3.50,3.58
2,Witbier,5.4,3.75,3.25,3.75,3.50,3.50,3.48


In [ ]:
#sorting out categorical variables (beer styles) using a frequency encoder
# Calculate frequency of each category in the 'beer_style' column
# since it's categorical and having 150+ beer styles causes high cardinality so we dont one hot encode
frequency_encoding = X['style'].value_counts() / len(df)

# Map each category to its frequency
X['encoded_beer_style'] = X['style'].map(frequency_encoding)
X = X.drop(columns=['style'])
# Now, df['encoded_beer_style'] contains the frequency of each beer style
X.head(3)


,abv,appearance,aroma,palate,taste,overall,rating,encoded_beer_style
0,4.4,2.50,4.00,4.00,3.75,3.75,3.76,0.005233
1,4.6,3.00,3.75,4.00,3.50,3.50,3.58,0.003188
2,5.4,3.75,3.25,3.75,3.50,3.50,3.48,0.014050


In [22]:
# Step 3: Train a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y_encoded)

# Step 4: Get feature importances
feature_importances = rf.feature_importances_
feature_names = X.columns

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

# Display the top features
print(importance_df.head(10))

# Step 5: Visualize feature importances
plt.figure(figsize=(10, 8))
sns.barplot(data=importance_df.head(10), x='Importance', y='Feature', palette='viridis')
plt.title('Top 10 Feature Importances')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.show()

MemoryError: could not allocate 1677721600 bytes